# **NLP Assignment 9**: Naive Bayes Modeling Method
#### Emily Daskas


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

A small sample of 2 variables distribution is given as follows. There are 9 observations. 

The domain of A variable is = A1, A2, A3 

The domain of B variable is = B1, B2, B3, B4, B5

Compute the given probabilities.

In [ ]:
A = ['A1','A2','A2','A1','A3','A3','A1','A3','A1']
B = ['B1','B2','B2','B1','B4','B3','B4','B4','B3']
df = pd.DataFrame(data = {"A":A,"B":B})
df 

,A,B
0,A1,B1
1,A2,B2
2,A2,B2
3,A1,B1
4,A3,B4
5,A3,B3
6,A1,B4
7,A3,B4
8,A1,B3


In [ ]:
s = df.groupby(['A', 'B']).size()
s = s.reindex(pd.MultiIndex.from_product(s.index.levels), fill_value=0)

In [ ]:
countsA = df.groupby('A').count()

In [ ]:
countsB = df.groupby('B').count()

In [ ]:
df_counts = pd.DataFrame(columns=["B1", "B2", "B3", "B4", "B5"], data = {"Total": countsA['B']})
df_counts = df_counts.append([pd.DataFrame(s['A1']).T, pd.DataFrame(s['A2']).T, pd.DataFrame(s['A3']).T])
df_counts = df_counts.append([countsB.T])
df_counts["B5"] = df_counts["B5"].replace(np.nan, 0)
df_counts['A'] = ['A1', 'A2', 'A3', 'B_totals']
df_counts = df_counts.set_index('A')
df_counts['A_totals'] = df_counts.sum(axis = 1)
df_counts


,B1,B2,B3,B4,B5,A_totals
A,,,,,,
A1,2,0,1,1,0,4.0
A2,0,2,0,0,0,2.0
A3,0,0,1,2,0,3.0
B_totals,2,2,2,3,0,9.0


In [ ]:
df_counts['B1']['A1']

2

In [ ]:
def reverse_probability(df, A, B):
  num_a_given_B = df[B][A]
  total = df[B]['B_totals']
  return num_a_given_B/total

In [ ]:
def compute_probabilities(df, A, B):

  reverse_prob = reverse_probability(df, A, B)


  total_B = df[B]['B_totals']
  total = df['A_totals']['B_totals']

  numerator = reverse_prob * total_B / total


  total_A = df['A_totals'][A]
  denom = total_A/total

  return numerator/denom

0.5

In [ ]:
def compute_probabilities_laplace(df, A, B, k):

  reverse_prob = reverse_probability(df, A, B)

  total_B = df[B]['B_totals'] 
  total = df['A_totals']['B_totals']
  laplace_B = (total_B + k)/ (total + 5*k)
  numerator = reverse_prob * laplace_B

  total_A = df['A_totals'][A]
  laplace_A = (total_A + k)/(total + 3*k)

  return numerator/laplace_A

In [ ]:
A_list = ['A1', 'A2', 'A3']
B_list = ['B1', 'B2', 'B3', 'B4', 'B5']


In [ ]:

print("PROBABILTIES")
print()
for a in A_list:
  for b in B_list:
    print("B =", b, "| A =", a, ":", compute_probabilities(df_counts, a, b))
  print()

PROBABILTIES

B = B1 | A = A1 : 0.5
B = B2 | A = A1 : 0.0
B = B3 | A = A1 : 0.25
B = B4 | A = A1 : 0.25
B = B5 | A = A1 : nan

B = B1 | A = A2 : 0.0
B = B2 | A = A2 : 1.0
B = B3 | A = A2 : 0.0
B = B4 | A = A2 : 0.0
B = B5 | A = A2 : nan

B = B1 | A = A3 : 0.0
B = B2 | A = A3 : 0.0
B = B3 | A = A3 : 0.3333333333333333
B = B4 | A = A3 : 0.6666666666666666
B = B5 | A = A3 : nan



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


In [ ]:

print("LAPLACE SMOOTHING")
print()
for a in A_list:
  for b in B_list:
    print("B =", b, "| A =", a, ":", compute_probabilities_laplace(df_counts, a, b, 1))
  print()

LAPLACE SMOOTHING

B = B1 | A = A1 : 0.5142857142857142
B = B2 | A = A1 : 0.0
B = B3 | A = A1 : 0.2571428571428571
B = B4 | A = A1 : 0.22857142857142854
B = B5 | A = A1 : nan

B = B1 | A = A2 : 0.0
B = B2 | A = A2 : 0.8571428571428571
B = B3 | A = A2 : 0.0
B = B4 | A = A2 : 0.0
B = B5 | A = A2 : nan

B = B1 | A = A3 : 0.0
B = B2 | A = A3 : 0.0
B = B3 | A = A3 : 0.32142857142857145
B = B4 | A = A3 : 0.5714285714285714
B = B5 | A = A3 : nan



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


# Problem 2

In [ ]:
corpus = [["money, money, money", 'Spam'], ["free money for free gambling fun", 'Spam'], ["gambling for fun", 'Spam'], ["machine learning for fun, fun, fun", 'Not'], ["free machine learning", 'Not']]

In [ ]:
corpus_df = pd.DataFrame(corpus, columns=['Sentence', 'Class'])

In [ ]:
corpus_df

,Sentence,Class
0,"money, money, money",Spam
1,free money for free gambling fun,Spam
2,gambling for fun,Spam
3,"machine learning for fun, fun, fun",Not
4,free machine learning,Not


In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(corpus_df['Sentence'])

In [ ]:
print(X)

  (0, 6)	3
  (1, 6)	1
  (1, 1)	2
  (1, 0)	1
  (1, 3)	1
  (1, 2)	1
  (2, 0)	1
  (2, 3)	1
  (2, 2)	1
  (3, 0)	1
  (3, 2)	3
  (3, 5)	1
  (3, 4)	1
  (4, 1)	1
  (4, 5)	1
  (4, 4)	1


In [ ]:
df = pd.DataFrame(X.toarray(),columns=cv.get_feature_names())
df['Class'] = corpus_df['Class']

In [ ]:
df

,for,free,fun,gambling,learning,machine,money,Class
0,0,0,0,0,0,0,3,Spam
1,1,2,1,1,0,0,1,Spam
2,1,0,1,1,0,0,0,Spam
3,1,0,3,0,1,1,0,Not
4,0,1,0,0,1,1,0,Not


In [ ]:
len_corpus = len(corpus)
p = df.where(df['Class'] == 'Spam').dropna()
p_true = len(p)/len_corpus
f = df.where(df['Class'] == 'Not').dropna()
p_negative = len(f)/len_corpus

print("FIRST LEVEL PROBABILITIES:")
print("True (Spam) Documents =", p_true)
print("False (Not Spam) Documents =", p_negative)

FIRST LEVEL PROBABILITIES:
True (Spam) Documents = 0.6
False (Not Spam) Documents = 0.4


In [ ]:
spam_df = df.where(df['Class'] == 'Spam').dropna()
sum_of_cols = spam_df.sum(axis = 0)

spam_df_sum = spam_df.append(sum_of_cols, ignore_index = True)
spam_df_sum.at[3, 'Class'] = 'SUM Spam'
print(spam_df_sum)

   for  free  fun  gambling  learning  machine  money     Class
0  0.0   0.0  0.0       0.0       0.0      0.0    3.0      Spam
1  1.0   2.0  1.0       1.0       0.0      0.0    1.0      Spam
2  1.0   0.0  1.0       1.0       0.0      0.0    0.0      Spam
3  2.0   2.0  2.0       2.0       0.0      0.0    4.0  SUM Spam


In [ ]:
vocabulary = len(cv.get_feature_names())

In [ ]:
total_spam_words = 0
for i in range(vocabulary):
  total_spam_words = total_spam_words + sum_of_cols[i]
conditional_probability = sum_of_cols

for i in range(vocabulary):
  conditional_probability[i] = (conditional_probability[i] + 1) / (vocabulary + total_spam_words)

df_spam_sum_prob = spam_df_sum.append(conditional_probability, ignore_index= True)
df_spam_sum_prob.at[4,'Class'] = 'Probability'

In [ ]:
print(df_spam_sum_prob)
df_spam_sum_prob.T

        for      free       fun  ...   machine     money        Class
0  0.000000  0.000000  0.000000  ...  0.000000  3.000000         Spam
1  1.000000  2.000000  1.000000  ...  0.000000  1.000000         Spam
2  1.000000  0.000000  1.000000  ...  0.000000  0.000000         Spam
3  2.000000  2.000000  2.000000  ...  0.000000  4.000000     SUM Spam
4  0.157895  0.157895  0.157895  ...  0.052632  0.263158  Probability

[5 rows x 8 columns]


,0,1,2,3,4
for,0,1,1,2,0.157895
free,0,2,0,2,0.157895
fun,0,1,1,2,0.157895
gambling,0,1,1,2,0.157895
learning,0,0,0,0,0.0526316
machine,0,0,0,0,0.0526316
money,3,1,0,4,0.263158
Class,Spam,Spam,Spam,SUM Spam,Probability


In [ ]:
not_spam_df = df.where(df['Class'] == 'Not').dropna()
sum_of_cols = not_spam_df.sum(axis = 0)

not_spam_df_sum = not_spam_df.append(sum_of_cols, ignore_index = True)
not_spam_df_sum.at[2, 'Class'] = 'SUM Non-Spam'
print(not_spam_df_sum)

   for  free  fun  gambling  learning  machine  money         Class
0  1.0   0.0  3.0       0.0       1.0      1.0    0.0           Not
1  0.0   1.0  0.0       0.0       1.0      1.0    0.0           Not
2  1.0   1.0  3.0       0.0       2.0      2.0    0.0  SUM Non-Spam


In [ ]:
total_not_spam_words = 0
for i in range(vocabulary):
  total_not_spam_words = total_not_spam_words + sum_of_cols[i]
conditional_probability = sum_of_cols

for i in range(vocabulary):
  conditional_probability[i] = (conditional_probability[i] + 1) / (vocabulary + total_not_spam_words)

df_not_spam_sum_prob = not_spam_df_sum.append(conditional_probability, ignore_index= True)
df_not_spam_sum_prob.at[3,'Class'] = 'Probability'

In [ ]:
print(df_not_spam_sum_prob)

        for      free      fun  ...   machine     money         Class
0  1.000000  0.000000  3.00000  ...  1.000000  0.000000           Not
1  0.000000  1.000000  0.00000  ...  1.000000  0.000000           Not
2  1.000000  1.000000  3.00000  ...  2.000000  0.000000  SUM Non-Spam
3  0.140625  0.140625  0.15625  ...  0.148438  0.132812   Probability

[4 rows x 8 columns]


In [ ]:
df_not_spam_sum_prob.T

,0,1,2,3
for,1,0,1,0.140625
free,0,1,1,0.140625
fun,3,0,3,0.15625
gambling,0,0,0,0.132812
learning,1,1,2,0.148438
machine,1,1,2,0.148438
money,0,0,0,0.132812
Class,Not,Not,SUM Non-Spam,Probability


In [ ]:
def prob_sum(text, df, prob):
  prob_sum = 0
  for i in (df):
    for ii in text:
      if ii == i:
        prob_sum = df[i][len(df)-1]
  return prob * prob_sum

In [ ]:
test = "machine learning for free"
test = test.split(' ')
test

['machine', 'learning', 'for', 'free']

In [ ]:
prob_sum(test, df_spam_sum_prob, p_true) #𝑃𝑟𝑜𝑏𝑎𝑏𝑖𝑙𝑖𝑡𝑦_𝐷𝑜𝑐(𝑆𝑃𝐴𝑀=𝑡𝑟𝑢𝑒)

0.03157894736842105

In [ ]:

prob_sum(test, df_not_spam_sum_prob, p_negative) #𝑃𝑟𝑜𝑏𝑎𝑏𝑖𝑙𝑖𝑡𝑦_𝐷𝑜𝑐(𝑆𝑃𝐴𝑀=𝑓𝑎𝑙𝑠𝑒)

0.059375000000000004

In [ ]:
print("Is the text spam?")
prob_sum(test, df_spam_sum_prob, p_true) > prob_sum(test, df_not_spam_sum_prob, p_negative)

Is the text spam?


False